In [2]:
from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession

config = ConfigProto()
config.gpu_options.allow_growth = True


In [3]:
import numpy as np
import pandas as pd
import math
import os
import scipy.misc
import matplotlib.pyplot as plt 
from tensorflow.keras.utils import to_categorical
%matplotlib inline 
# from keras import keras.layer

In [4]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
# from tensorflow.keras.utils import np_utils
from tensorflow.keras.preprocessing.image import ImageDataGenerator 
from tensorflow.keras.utils import plot_model
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Flatten, Dropout, BatchNormalization
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import concatenate
from tensorflow.keras.optimizers import Adam, SGD
from tensorflow.keras.regularizers import l1, l2
from sklearn.metrics import confusion_matrix
import itertools

In [6]:
dataset=pd.read_csv("../deb")

FileNotFoundError: [Errno 2] No such file or directory: '../data/fer2013.csv'

In [ ]:
dataset.describe()

In [ ]:
for i in range (1,7):
    print(dataset.emotion[i])
    array = np.mat(dataset.pixels[i]).reshape(48,48) 
    plt.imshow(array,cmap='gray')
    plt.show()


In [ ]:
dataset.head()

In [ ]:
#X,y=dataset['pixels'],dataset['emotion']
dataset.count()

<b>we dont have any missing value</b>

In [ ]:
dataset.groupby('Usage').size()

In [ ]:
train_ds=dataset[dataset['Usage']=='Training']
test_ds=dataset[dataset['Usage']=='PrivateTest']
val_ds=dataset[dataset['Usage']=='PublicTest']


In [ ]:
test_ds.count()

In [ ]:
val_ds.count()

In [ ]:
train_ds.count()

In [ ]:
type(dataset.pixels[0])

In [ ]:
X_train= np.array(list(map(str.split, train_ds.pixels)), np.float32)
X_test=np.array(list(map(str.split,test_ds.pixels)),np.float32)
X_val=np.array(list(map(str.split,val_ds.pixels)),np.float32)

In [ ]:
image_size=int(math.sqrt(X_train[0].shape[0]))# =48 calculate  original image width_hight

In [ ]:
# image shape as (H, W, D)
X_train=X_train.reshape((X_train.shape[0],image_size,image_size,1)) # generate (48,48,1) images
X_test=X_test.reshape((X_test.shape[0],image_size,image_size,1))
X_val=X_val.reshape((X_val.shape[0],image_size,image_size,1))

In [ ]:
type(dataset.emotion[0])

In [ ]:
num_classes = 7

y_train=train_ds.emotion
y_train = to_categorical(y_train, num_classes) 
y_test=test_ds.emotion
y_test=to_categorical(y_test,num_classes)
y_val=val_ds.emotion
y_val=to_categorical(y_val,num_classes)

<h2>Vgg19 model</h2>

<a><img src='./vgg16.png' title='vgg16' ></a>

In [ ]:
def FER_Model(input_shape=(48,48,1)):
    # first input model
    visible = Input(shape=input_shape, name='input')
    num_classes = 7
    #the 1-st block
    conv1_1 = Conv2D(64, kernel_size=3, activation='relu', padding='same', name = 'conv1_1')(visible)
    conv1_1 = BatchNormalization()(conv1_1)
    conv1_2 = Conv2D(64, kernel_size=3, activation='relu', padding='same', name = 'conv1_2')(conv1_1)
    conv1_2 = BatchNormalization()(conv1_2)
    pool1_1 = MaxPooling2D(pool_size=(2,2), name = 'pool1_1')(conv1_2)
    drop1_1 = Dropout(0.3, name = 'drop1_1')(pool1_1)

    #the 2-nd block
    conv2_1 = Conv2D(128, kernel_size=3, activation='relu', padding='same', name = 'conv2_1')(drop1_1)
    conv2_1 = BatchNormalization()(conv2_1)
    conv2_2 = Conv2D(128, kernel_size=3, activation='relu', padding='same', name = 'conv2_2')(conv2_1)
    conv2_2 = BatchNormalization()(conv2_2)
    conv2_3 = Conv2D(128, kernel_size=3, activation='relu', padding='same', name = 'conv2_3')(conv2_2)
    conv2_3 = BatchNormalization()(conv2_3)
    pool2_1 = MaxPooling2D(pool_size=(2,2), name = 'pool2_1')(conv2_3)
    drop2_1 = Dropout(0.3, name = 'drop2_1')(pool2_1)

    #the 3-rd block
    conv3_1 = Conv2D(256, kernel_size=3, activation='relu', padding='same', name = 'conv3_1')(drop2_1)
    conv3_1 = BatchNormalization()(conv3_1)
    conv3_2 = Conv2D(256, kernel_size=3, activation='relu', padding='same', name = 'conv3_2')(conv3_1)
    conv3_2 = BatchNormalization()(conv3_2)
    conv3_3 = Conv2D(256, kernel_size=3, activation='relu', padding='same', name = 'conv3_3')(conv3_2)
    conv3_3 = BatchNormalization()(conv3_3)
    conv3_4 = Conv2D(256, kernel_size=3, activation='relu', padding='same', name = 'conv3_4')(conv3_3)
    conv3_4 = BatchNormalization()(conv3_4)
    pool3_1 = MaxPooling2D(pool_size=(2,2), name = 'pool3_1')(conv3_4)
    drop3_1 = Dropout(0.3, name = 'drop3_1')(pool3_1)

    #the 4-th block
    conv4_1 = Conv2D(256, kernel_size=3, activation='relu', padding='same', name = 'conv4_1')(drop3_1)
    conv4_1 = BatchNormalization()(conv4_1)
    conv4_2 = Conv2D(256, kernel_size=3, activation='relu', padding='same', name = 'conv4_2')(conv4_1)
    conv4_2 = BatchNormalization()(conv4_2)
    conv4_3 = Conv2D(256, kernel_size=3, activation='relu', padding='same', name = 'conv4_3')(conv4_2)
    conv4_3 = BatchNormalization()(conv4_3)
    conv4_4 = Conv2D(256, kernel_size=3, activation='relu', padding='same', name = 'conv4_4')(conv4_3)
    conv4_4 = BatchNormalization()(conv4_4)
    pool4_1 = MaxPooling2D(pool_size=(2,2), name = 'pool4_1')(conv4_4)
    drop4_1 = Dropout(0.3, name = 'drop4_1')(pool4_1)

    #the 5-th block
    conv5_1 = Conv2D(512, kernel_size=3, activation='relu', padding='same', name = 'conv5_1')(drop4_1)
    conv5_1 = BatchNormalization()(conv5_1)
    conv5_2 = Conv2D(512, kernel_size=3, activation='relu', padding='same', name = 'conv5_2')(conv5_1)
    conv5_2 = BatchNormalization()(conv5_2)
    conv5_3 = Conv2D(512, kernel_size=3, activation='relu', padding='same', name = 'conv5_3')(conv5_2)
    conv5_3 = BatchNormalization()(conv5_3)
    conv5_4 = Conv2D(512, kernel_size=3, activation='relu', padding='same', name = 'conv5_4')(conv5_3)
    conv5_4 = BatchNormalization()(conv5_4)
    pool5_1 = MaxPooling2D(pool_size=(2,2), name = 'pool5_1')(conv5_4)
    drop5_1 = Dropout(0.3, name = 'drop5_1')(pool5_1)

    #Flatten and output
    flatten = Flatten(name = 'flatten')(drop5_1)
    ouput = Dense(num_classes, activation='softmax', name = 'output')(flatten)

    # create model 
    model = Model(inputs =visible, outputs = ouput)
    # summary layers
    print(model.summary())
    
    return model

In [ ]:
model = FER_Model()

In [ ]:
opt=tf.keras.optimizers.Adam()
loss=tf.keras.losses.CategoricalCrossentropy()
model.compile(optimizer=opt,loss=loss,metrics=['accuracy'])


<p style="color:red;"> model with out data generator</p>

In [ ]:
checkpoint_path = "./training_checkpoints/"
! mkdir -p {checkpoint_path}
checkpoint_dir2 = os.path.join(checkpoint_path,"ckpt_{epoch}")
cp_callback2 = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_dir,
                                                 save_weights_only=True,
                                                 verbose=1)



In [ ]:
history2=model.fit(train_flow,
                   batch_size=batch_size,
                   epochs=100,
                   callbacks=[cp_callback2],
                   validation_data=val_flow)

In [ ]:
plt.plot(history.history['accuracy'], label='accuracy')
plt.plot(history.history['val_accuracy'], label = 'val_accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
# plt.ylim([0.5, 1])
plt.legend(loc='lower right')

#test_loss, test_acc = model.evaluate(test_images,  test_labels, verbose=2)
plt.savefig("datagen.png")

In [ ]:
!mkdir -p vgg_model
model.save('saved_model/my_model') 